# M2GARSS 2024 Tutorial: Quantum Computing for Earth Observation
## Example 1: Acquisition planning for EO satellites

In this example, we test two different quantum optimization algorithms: QAOA and quantum annealing. We consider the acquisition planning problem as test application.

A straightforward, high-level library for generating problem instances and testing QAOA is available.
It is called `mqt-problemsolver` (it is already included in the `requirements.txt` file).
Check out their [documentation](https://github.com/cda-tum/mqt-problemsolver) and their paper:

*Quetschlich, N., Koch, V., Burgholzer, L., & Wille, R. (2023). A Hybrid Classical Quantum Computing Approach to the Satellite Mission Planning Problem. https://arxiv.org/abs/2308.00029v1*


In [ ]:
import sys
sys.path.append("..")
import example1

import numpy as np
import pytest
from mqt.problemsolver.satellitesolver import algorithms, utils
from mqt.problemsolver.satellitesolver.evaluator import eval_all_instances_Satellite_Solver
from mqt.problemsolver.satellitesolver.ImagingLocation import LocationRequest
from qiskit_optimization import QuadraticProgram
from typing import Any

from dwave.cloud import Client
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dimod
import dwave_networkx as dnx
import networkx as nx
import matplotlib.pyplot as plt

### Problem instance

In [ ]:
num_to_be_captured_locations = 5
ac_requests = utils.init_random_acquisition_requests(num_to_be_captured_locations)
converter, qubo = utils.convert_docplex_to_qubo(utils.create_satellite_doxplex(ac_requests))

### Quantum Approximate Optimization Algorithm (QAOA)
For QAOA, a real or simulated quantum platform for running quantum circuit is required. [IBM Quantum](https://quantum.ibm.com/) offers free cloud access to quantum computers based on superconducting qubits. In practice, simulating quantum circuits on a personal computer for testing purposes is much more time efficient. For this, the [Qiskit](https://www.ibm.com/quantum/qiskit) library can be used.

In [ ]:
from mqt.problemsolver.satellitesolver import utils
from mqt.problemsolver.satellitesolver.algorithms import solve_using_qaoa

use_noisy_backend = True
res_qaoa = solve_using_qaoa(qubo, noisy_flag=use_noisy_backend)


### Quantum Annealing
For running quantum annealing, access to a D-Wave quantum annealer has to be obtained. A free trial account on [D-Wave Leap](https://cloud.dwavesys.com/leap/) can be created. Then, here are instructions on [how to configure access to the D-Wave solvers](https://docs.ocean.dwavesys.com/en/stable/overview/sapi.html).

In [ ]:
def solve_using_qa(qubo, qa_setup: dict[str, Any]):
    # Create a client instance
    client = Client.from_config()

    # Get the list of available solvers
    solvers = client.get_solvers()

    # Print the name of the used solvers
    print("Using solver", solvers[0].id)

    # Create a sampler
    sampler = DWaveSampler(solver=solvers[0].id)

    # Solve the QUBO using the D-Wave sampler
    sampleset = sampler.sample_qubo(qubo, num_reads=qa_setup("num_reads"), label='Acquisition Planning')
    return sampleset